In [1]:
import dash
import copy
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_table as dt
import numpy as np
import os
from sklearn.svm import SVC
import pandas as pd
import plotly
import time
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State
from flask import Flask
from plotly import graph_objs as go
from plotly.graph_objs import *
import dash_table_experiments as dt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import base64
import datetime
import io
import dash_table

In [2]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

In [3]:
Data_Taiwan= pd.read_csv("../Data/Data_Taiwan_final.csv")
Data_america = pd.read_csv("../Data/america_data.csv")
Data_german = pd.read_csv("../Data/Data_German_final.csv")

In [5]:
df_t=pd.read_excel('../Data/default of credit card clients.xls',skiprows=1,index_col=0)


In [6]:
df=pd.read_csv('../Data/Bank_of_America_data.csv')
df = df.dropna()



available_indicators = df['JOB'].unique()

#creation des listes:
L2=[1,2,3,4]
effectifBAD_taiwan=[]
effectifGOOD_taiwan=[]
for i in range(len(L2)):
    effectifBAD_taiwan.append(df_t[( df_t['EDUCATION']==L2[i] ) & ( df_t['default payment next month'] ==1 )].shape[0])
for i in range(len(L2)):
    effectifGOOD_taiwan.append(df_t[( df_t['EDUCATION']==L2[i] ) & ( df_t['default payment next month'] ==0 )].shape[0])

In [7]:
L=[list(range(19, 31)),list(range(31, 41)),list(range(41, 51)),list(range(51, 61)),list(range(61, 71)),list(range(71, 81))]
effectifBAD=[]
effectifGOOD=[]
for i in range(len(L)):
    effectifBAD.append(Data_german[( Data_german['Age in years'].isin(L[i]) ) & ( Data_german['target'] ==0 )].shape[0])
for i in range(len(L)):
    effectifGOOD.append(Data_german[( Data_german['Age in years'].isin(L[i]) ) & ( Data_german['target'] ==1 )].shape[0])

In [8]:
# data taiwan
#df = Data_Taiwan.copy()

In [9]:
StandardScalerT = StandardScaler()

In [10]:
X = Data_Taiwan.iloc[:,:-1]
Y = Data_Taiwan.iloc[:,-1]
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, random_state = 0)

In [11]:
X_train = StandardScalerT.fit_transform(X_train)
X_test=StandardScalerT.transform(X_test)

In [12]:
model_KNN = KNeighborsClassifier(n_neighbors=8)
model_KNN.fit(X_train, Y_train)
acc_KNN = model_KNN.score(X_test, Y_test)
y_pred_knn =model_KNN.predict(X_test)
acc_KNN

0.8185380354006215

In [13]:
# data america

In [14]:
from sklearn.preprocessing import StandardScaler

StandardScalerA = StandardScaler()

In [15]:
X_america = Data_america.iloc[:,1:]
Y_america = Data_america.iloc[:,0]
X_train_america, X_test_america, Y_train_america,Y_test_america = train_test_split(X_america,Y_america, random_state = 0)

In [16]:
X_train_america=StandardScalerA.fit_transform(X_train_america)
X_test_america=StandardScalerA.transform(X_test_america)

In [17]:
model_KNN_america = KNeighborsClassifier(n_neighbors=3)
model_KNN_america.fit(X_train_america, Y_train_america)
acc_KNN_america = model_KNN_america.score(X_test_america, Y_test_america)
y_pred_knn_america =model_KNN_america.predict(X_test_america)
acc_KNN_america

0.9026028547439127

In [18]:
from sklearn.preprocessing import StandardScaler

StandardScalerG = StandardScaler()

In [19]:
X_german = Data_german.iloc[:,:-1]
Y_german = Data_german.iloc[:,-1]
X_train_german, X_test_german, Y_train_german,Y_test_german = train_test_split(X_german,Y_german, random_state = 0)

In [20]:
X_train_german=StandardScalerG.fit_transform(X_train_german)
X_test_german=StandardScalerG.transform(X_test_german)

In [21]:
svclassifier4 = SVC(kernel='sigmoid')
svclassifier4.fit(X_train_german, Y_train_german)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [22]:
y_pred_SVM_german = svclassifier4.predict(X_test_german)

In [23]:
#model_KNN_german = KNeighborsClassifier(n_neighbors=4)
#model_KNN_german.fit(X_train_german, Y_train_german)
#acc_KNN_german = model_KNN_german.score(X_test_german, Y_test_german)
#y_pred_knn_german =model_KNN_german.predict(X_test_german)
#acc_KNN_german

In [24]:
acc_SVM_german = svclassifier4.score(X_test_german, Y_test_german)
acc_SVM_german

0.748

In [25]:
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)
app.layout = html.Div([

      
    html.Div(html.H1(children="Scoring d'un client d'une banque en vue de l'octroi d'un crédit",
                     style={'text-align':'center',
                            'color':'black',
                            'background-color':'#1aa3ff',
                            'margin-left':"15px",
                            'margin-top':"15px",
                            'padding':'10px'
                           })),
    #html.Label('choose your bank : '),
    html.Div(dcc.Dropdown(
        id='show',
        value='Choose your bank',
        style={'margin-top':"15px",'margin-left':'10px'},
        options=[
            {'label': 'Choose your bank', 'value': 'Choose your bank'},
            {'label': 'Bank of America', 'value': 'AMR'},
            {'label': 'Bank of German', 'value': 'GER'},
            {'label': 'Bank of Taiwan', 'value': 'TWN'}
        ]
    )),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    
    dcc.Tabs(id="tabs_taiwan",style={'display':'none'} ,children=[
         dcc.Tab(label='figure1', children=[
         dcc.Graph(
    id='taiwan-graph2',
    figure={
        'data' : [
            {
                'values': [22,78],
                'type': 'pie',
            },
        ],
        'layout': {
            'margin': {
            'l': 30,
            'r': 0,
            'b': 30,
            't': 0
            },
            'legend': {'x': 0, 'y': 1}
        }
    }
),]),
    dcc.Tab(label='figure2', children=[
    dcc.Graph(
        id='taiwan-graph1',
        figure={
            'data': [
                {'x': L2, 'y': effectifGOOD_taiwan, 'type': 'bar','name': 'Good'},
                {'x': L2, 'y': effectifBAD_taiwan, 'type': 'bar','name': 'Bad'}
            ],
            'layout': {
                'title': 'Target vs education level'
            }
        }
    )])]),
    
    html.Div(className='row',id='form_taiwan',style={'display':'none'},
             children=[
    html.Label('Personal informations',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="First name",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'},
                 id='firstnamet'),
    dcc.Input(type='text',placeholder="Last name",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'},
             id="lastnamet"),
    
    
    dcc.Input(type='text',placeholder="Age",id="age",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    
    
    dcc.Dropdown(
        value='Gender',
        id ='gender',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Choose your Gender',
        options=[
            {'label': 'Male', 'value': '1'},
            {'label': 'Female', 'value': '2'}
        ]
    ),
    
    dcc.Dropdown(
        value='Education',
        id ='education',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Choose your Education',
        options=[
            {'label': 'graduate school', 'value': 'graduate_school'},
            {'label': 'University', 'value': 'university'},
            {'label': 'High school', 'value': 'high school'},
            {'label': 'Others', 'value': 'other_education'}
        ]
    ),
        
    dcc.Dropdown(
        value='Marital status',
        id ='statut',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Choose your Marital status',
        options=[
            {'label': 'married', 'value': 'married'},
            {'label': 'single', 'value': 'single'},
            {'label': 'Others', 'value': 'other_mariage'}
        ]
    ),
      
    html.Label('Amount of the given credit (NT dollar)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Amount of the given credit",id="limit",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),             
                 
                 
    html.Label('the repayment status in September',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in September',
        id='repaySeptember',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in September',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_01'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_02'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_03'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_04'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_05'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_06'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_07'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_08'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_09'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_10'}
        ]
    ),
    
    html.Label('the repayment status in August',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in August',
        id='repayAugust',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in August',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_2_1'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_2_2'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_2_3'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_2_4'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_2_5'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_2_6'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_2_7'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_2_8'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_2_9'},
            {'label': 'Payment delay for 9 month', 'value': 'PAY_2_10'}
        ]
    ),
                 
    html.Label('the repayment status in July',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in July',
        id='repayJuly',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in July',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_3_1'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_3_2'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_3_3'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_3_4'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_3_5'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_3_6'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_3_7'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_3_8'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_3_9'},
            {'label': 'Payment delay for 9 month', 'value': 'PAY_3_10'}
        ]
    ),
      
                 
    html.Label('the repayment status in June',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in June',
        id='repayJune',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in June',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_4_1'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_4_2'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_4_3'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_4_4'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_4_5'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_4_6'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_4_7'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_4_8'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_4_9'},
            {'label': 'Payment delay for 9 month', 'value': 'PAY_4_10'}
        ]
    ),
     
    html.Label('the repayment status in May',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in May',
        id='repayMay',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in May',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_5_1'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_5_2'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_5_3'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_5_4'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_5_5'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_5_6'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_5_7'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_5_8'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_5_9'}
        ]
    ),
      
    html.Label('the repayment status in April',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        value='the repayment status in April',
        id='repayApril',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='the repayment status in April',
        options=[
            {'label': 'Pay duly', 'value': 'PAY_6_1'},
            {'label': 'Payment delay for 1 month', 'value': 'PAY_6_2'},
            {'label': 'Payment delay for 2 month', 'value': 'PAY_6_3'},
            {'label': 'Payment delay for 3 month', 'value': 'PAY_6_4'},
            {'label': 'Payment delay for 4 month', 'value': 'PAY_6_5'},
            {'label': 'Payment delay for 5 month', 'value': 'PAY_6_6'},
            {'label': 'Payment delay for 6 month', 'value': 'PAY_6_7'},
            {'label': 'Payment delay for 7 month', 'value': 'PAY_6_8'},
            {'label': 'Payment delay for 8 month', 'value': 'PAY_6_9'},
        ]
    ),
                 
      
                 
    html.Label('Amount of bill statement from April to September(NT dollar)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in April",id="billapril",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in May",id="billmay",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in June",id="billjune",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in July",id="billjuly",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in August",id="billaugust",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Amount of bill statement in September",id="billseptember",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Amount of previous payment from April to September(NT dollar)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="amount paid in April",id="payapril",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),     
    dcc.Input(type='text',placeholder="amount paid in May",id="paymay",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="amount paid in June",id="payjune",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="amount paid in July",id="payjuly",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),     
    dcc.Input(type='text',placeholder="amount paid in August",id="payaugust",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="amount paid in Septembe",id="payseptember",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),             
                 
                 
                 
   
    html.Button('Click here to see the result',style={'margin-left':'35%','margin-top':"20px"},id="submit"),
               
        ],

    ),
    dcc.Tabs(id="tabs_america" ,style={'display': 'none'},children=[
        dcc.Tab(label='figure1', children=[
     html.Div([        
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value=''
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value=''
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': ''}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['BAD'].min(),
        max=df['BAD'].max(),
        value=df['BAD'].max(),
        marks={str(year): str(year) for year in df['BAD'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})],style={'display': 'none'},id="taab")]),
    dcc.Tab(label='figure1', children=[
        
        dcc.Graph(
    id='america-graph',
    figure={
        'data' : [
            {
                'values': [20,80],
                'type': 'pie',
            },
        ],
        'layout': {
            'margin': {
            'l': 30,
            'r': 0,
            'b': 30,
            't': 0
            },
            'legend': {'x': 0, 'y': 1}
        }
    }
)
        
        
        
    
   ])]),
    
    html.Div(className='row',id='form_america',style={'display':'none'},
             children=[
    html.Label('Personal informations',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="firstname",id="firstnamea",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="lastname",id="lastnamea",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    
    html.Label('Reason',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="reason",
        value={'Reason':'5px'},
        placeholder='Choose your Reason',
        options=[
            {'label': 'Home improvement', 'value': '1'},
            {'label': 'Debt consolidation', 'value': '0'}
        ]
    ),
    
    html.Label('Job',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="job",
        value='Job',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Choose your Job',
        options=[
            {'label': 'Office', 'value': 'Office'},
            {'label': 'Sales', 'value': 'Sales'},
            {'label': 'Mgr', 'value': 'Mgr'},
            {'label': 'ProfExe', 'value': 'ProfExe'},
            {'label': 'Self', 'value': 'Self'},
            {'label': 'Other', 'value': 'Other'},
        ]
    ),
    
    html.Label('Amount of the loan request (LOAN)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Amount of the loan request",id="LOAN",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Amount due on existing mortgage (MORTDUE)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Amount due on existing mortgage",id="MORTDUE",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}), 
    
    html.Label('Value of current property (VALUE)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Value of current property",id="VALUE",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Years at present job (YOJ)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Years at present job",id="YOJ",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),             
                 
    html.Label('Number of major derogatory reports (DEROG)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of major derogatory reports",id="DEROG",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}), 
    
    html.Label('Number of delinquent credit lines (DELINQ)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of delinquent credit lines",id="DELINQ",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Age of oldest trade line in months (CLAGE)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Age of oldest trade line in months",id="CLAGE",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Number of recent credit lines (NINQ)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of recent credit lines",id="NINQ",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),        
    
    html.Label('Number of credit lines(CLNO)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of credit lines",id="CLNO",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Debt-to-income ratio (DEBTINC)',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Debt-to-income ratio",id="DEBTINC",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),

    html.Button('Click here to see the result',style={'margin-left':'35%','margin-top':"20px"},
                id="submit_america"),
   
                 
        ],

    ),
     dcc.Tabs(id="tabs",style={'display':'none'} ,children=[
         dcc.Tab(label='figure1', children=[
         dcc.Graph(
    id='german-graph2',
    figure={
        'data' : [
            {
                'values': [30,70],
                'type': 'pie',
            },
        ],
        'layout': {
            'margin': {
            'l': 30,
            'r': 0,
            'b': 30,
            't': 0
            },
            'legend': {'x': 0, 'y': 1}
        }
    }
),]),
    dcc.Tab(label='figure2', children=[
    dcc.Graph(
        id='german-graph1',
        figure={
            'data': [
                {'x': L, 'y': effectifGOOD, 'type': 'bar','name': 'Good'},
                {'x': L, 'y': effectifBAD, 'type': 'bar','name': 'Bad'}
            ],
            'layout': {
                'title': 'Target vs age range'
            }
        }
    )])]),
    html.Div(className='row',id='form_german',style={'display':'none'},
             children=[
    html.Label('Personal informations',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="firstname",id="firstnameg",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="lastname",id="lastnameg",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    dcc.Input(type='text',placeholder="Age in years",id="Age in years",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    dcc.Dropdown(
        id="Telephone",
        value='Telephone',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Telephone',
        options=[
            {'label': 'A191 : none', 'value': '0'},
            {'label': 'A192 : yes, registered under the customers name', 'value': '1'},
        ]
    ),  
    dcc.Dropdown(
        id="Housing",
        value='Housing',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Housing',
        options=[
            {'label': 'A151 : rent', 'value': 'Housing_A151'},
            {'label': 'A152 : own', 'value': 'Housing_A152'},
           
        ]
    ),
    
    dcc.Dropdown(
        id="Foreign worker",
        value='Foreign worker',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Foreign worker',
        options=[
            {'label': 'A201 : yes', 'value': '1'},
            {'label': 'A202 : no', 'value': '0'},
        ]
    ),
                 
    dcc.Dropdown(
        id="Personal status",
        value='Personal status and sex',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Personal status and sex',
        options=[
            {'label': 'A91 : male   : divorced/separated', 'value': 'Personal status and sex_A91'},
            {'label': 'A92 : female : divorced/separated/married', 'value': 'Personal status and sex_A92'},
            {'label': 'A93 : male   : single', 'value': 'Personal status and sex_A93'},
            {'label': 'A94 : male   : married/widowed', 'value': 'Personal status and sex_A94'},
            {'label': 'A95 : female : single', 'value': 'Personal status and sex_A95'},
           
        ]
    ),
                 
    dcc.Dropdown(
        id="Job",
        value='Job',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Job',
        options=[
            {'label': 'A171 : unemployed/ unskilled  ', 'value': 'Job_A171'},
            {'label': 'A172 : unskilled - resident', 'value': 'Job_A172'},
            {'label': 'A173 : skilled employee / official', 'value': 'Job_A173'},
            {'label': 'A174 : management/ self-employed/', 'value': 'Job_A174'},
        ]
    ),
                 
                 
    html.Label('Status of existing checking account',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Status",
        value='Status of existing checking account',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Status of existing checking account',
        options=[
            {'label': 'A11 : ... <    0 DM', 'value': 'Status_A11'},
            {'label': 'A12 : 0 <= ... <  200 DM', 'value': 'Status_A12'},
            {'label': 'A13 :      ... >= 200 DM', 'value': 'Status_A13'},
            {'label': 'A14 : no checking account', 'value': 'Status_A14'}
        ]
    ),
    
    html.Label('Duration in month',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Duration in month",id="Duration",
    style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
                 
    html.Label('Credit history',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Credit history",
        value='Credit history',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Credit history',
        options=[
            {'label': 'A30 : no credits taken', 'value': 'Credit history_A30'},
            {'label': 'A31 : all credits at this bank paid back duly', 'value': 'Credit history_A31'},
            {'label': 'A32 : existing credits paid back duly till now', 'value': 'Credit history_A32'},
            {'label': 'A33 : delay in paying off in the past', 'value': 'Credit history_A33'},
            {'label': 'A34 : critical account', 'value': 'Credit history_A34'}
           
        ]
    ),
    
    html.Label('Purpose',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Purpose",
        value='Purpose',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Purpose',
        options=[
            {'label': 'A40 : car (new)', 'value': 'Purpose_A40'},
            {'label': 'A41 : car (used)', 'value': 'Purpose_A41'},
            {'label': 'A42 : furniture/equipment', 'value': 'Purpose_A42'},
            {'label': 'A43 : radio/television', 'value': 'Purpose_A43'},
            {'label': 'A44 : domestic appliances', 'value': 'Purpose_A44'},
            {'label': 'A45 : repairs', 'value': 'Purpose_A45'},
            {'label': 'A46 : education', 'value': 'Purpose_A46'},
            {'label': 'A48 : retraining', 'value': 'Purpose_A48'},
            {'label': 'A49 : business', 'value': 'Purpose_A49'},
            {'label': 'A410 : others', 'value': 'Purpose_A410'}
           
        ]
    ),
    
                 
    html.Label('Credit amount',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Credit amount",id="Credit amount",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
    
    html.Label('Savings account/bonds',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Savings account",
        value='Savings account/bonds',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Savings account/bonds',
        options=[
            {'label': 'A61 : ... <  100 DM', 'value': 'Savings account/bonds_A61'},
            {'label': 'A62 : 100 <= ... <  500 DM', 'value': 'Savings account/bonds_A62'},
            {'label': 'A63 : 500 <= ... < 1000 DM', 'value': 'Savings account/bonds_A63'},
            {'label': 'A64 : .. >= 1000 DM', 'value': 'Savings account/bonds_A64'},
            {'label': 'A65 : unknown/ no savings account', 'value': 'Savings account/bonds_A65'},
           
        ]
    ),
    
    
    html.Label('Present employment since',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Present employment",
        value='Present employment since',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Present employment since',
        options=[
            {'label': 'A71 : unemployed', 'value': 'Present employment since_A71'},
            {'label': 'A72 : ... < 1 year', 'value': 'Present employment since_A72'},
            {'label': 'A73 : 1  <= ... < 4 years', 'value': 'Present employment since_A73'},
            {'label': 'A74 : 4  <= ... < 7 years', 'value': 'Present employment since_A74'},
            {'label': 'A75 : .. >= 7 years', 'value': 'Present employment since_A75'},
           
        ]
    ),
                 
                 
    html.Label('Installment rate in percentage of disposable income',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Installment rate in percentage of disposable income",id="Installment rate",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}), 
    
    
     
    html.Label('Other debtors / guarantors',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Other debtors / guarantors",
        value='Other debtors / guarantors',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Other debtors / guarantors',
        options=[
            {'label': 'A101 : none', 'value': 'Other debtors / guarantors_A101'},
            {'label': 'A102 : co-applicant', 'value': 'Other debtors / guarantors_A102'},
            {'label': 'A103 : guarantor', 'value': 'Other debtors / guarantors_A103'},
           
        ]
    ),
                 
    html.Label('Present residence since',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Present residence since",id="Present residence since",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
     
    html.Label('Property',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Property",
        value='Property',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Property',
        options=[
            {'label': 'A121 : real estate', 'value': 'Property_A121'},
            {'label': 'A122 : life insurance', 'value': 'Property_A122'},
            {'label': 'A123 : car or other', 'value': 'Property_A123'},
            {'label': 'A124 : unknown / no property', 'value': 'Property_A124'},
        ]
    ),
                 
                 
    
    html.Label('Other installment plans',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Dropdown(
        id="Other installment plans",
        value='Other installment plans',
        style={'margin-top':"15px",'margin-left':'5px'},
        placeholder='Other installment plans',
        options=[
            {'label': 'A141 : bank', 'value': 'Other installment plans_A141'},
            {'label': 'A142 : stores', 'value': 'Other installment plans_A142'},
            {'label': 'A143 : none', 'value': 'Other installment plans_A143'}
        ]
    ),
      
                 
    html.Label('Number of existing credits at this bank',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of existing credits at this bank",id="Number of existing credits",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}), 
    
    
    html.Label('Number of people being liable to provide maintenance for',style={'margin-top':"15px","color":'#1aa3ff'}),
    dcc.Input(type='text',placeholder="Number of people being liable to provide maintenance for",id="Number of people being liable to provide maintenance for",
              style={'width': '100%', 'display': 'inline-block','margin-top':'15px','margin-left':'10px'}),
     
    
                 
    
    html.Button('Click here to see the result',style={'margin-left':'35%','margin-top':"20px"},id="submit_german"),      
        ],

    ),
    
     
     html.Div(className='row',style={'display':'none'},id="plot_taiwan",children=[
        #html.Button('Home',style={'margin-left':'35%','margin-top':"20px"},id="Home_taiwan"),
         html.Button('Please enter your informations',
                     style={'margin-left':'35%','margin-top':"20px"},
                     id="taiwan")]),
    
     html.Div(className='row',style={'display':'none'},id="plot_america",children=[
          #html.Button('Home',style={'margin-left':'35%','margin-top':"20px"},id="Home_america"),
         html.Button('Please enter your informations',style={'margin-left':'35%','margin-top':"20px"},id='america')]),
                    
    
     html.Div(className='row',style={'display':'none'},id="plot_german",children=[
          #html.Button('Home',style={'margin-left':'35%','margin-top':"20px"},id="Home_german"),
         html.Button('Please enter your informations',style={'margin-left':'35%','margin-top':"20px"},id='german')]),
                    
    
     html.Div(id='output-data-upload'),
     html.Div(html.H1(id="output_div",style={'margin':'20px'})),
     
     html.Div(html.H1(id="output_div_am",style={'margin':'20px'})),
     
     html.Div([html.H1(id="output_div_gr",style={'margin':'20px','background-image':'fg.jpg'}),
     ]),
    
    
])

def parse_data(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV or TXT file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'txt' or 'tsv' in filename:
            # Assume that the user upl, delimiter = r'\s+'oaded an excel file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')), delimiter = r'\s+')
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return df

       
    
@app.callback([ Output('german-graph1','style'),
                Output('taab','style'),
                Output('tabs_america','style'),
                Output('taiwan-graph2','style'),
                Output('tabs_taiwan','style'),
                Output('tabs','style'),
                Output('german-graph2','style'),
                Output('form_taiwan', 'style'),
                Output('form_america','style'),
                Output('form_german','style')], 
                [Input('show', 'value'),
                Input('submit', 'n_clicks'),
                Input('submit_america', 'n_clicks'),
                Input('submit_german', 'n_clicks')])


def show_plot(input_value,*click):
    if input_value == "TWN":
        if click[0] :
            return    {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'}
        return  {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'block'},{'display': 'block'},{'display': 'none'},{'display': 'none'},{'display': 'block'},{'display': 'none'},{'display': 'none'}
    elif input_value == "AMR":
        if click[1] :
            return   {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'}
        return  {'display': 'none'},{'display': 'block'},{'display': 'block'},{'display': 'none'}, {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'block'},{'display': 'none'}
    elif input_value == "GER":
        if click[2] :
            return   {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'}
        return   {'display': 'block'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'block'},{'display': 'block'},{'display': 'none'},{'display': 'none'},{'display': 'block'}
    else:
        return   {'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'},{'display': 'none'}

    
@app.callback(  Output('output_div', 'children'),
                [Input('submit', 'n_clicks')],
                [State('education', 'value'),
                State('statut', 'value'),
                State('repaySeptember', 'value'),
                State('repayAugust', 'value'),
                State('repayJuly', 'value'),
                State('repayJune', 'value'),
                State('repayMay', 'value'),
                State('repayApril', 'value'),
                State('limit', 'value'),
                State('gender', 'value'),
                State('age', 'value'),
                State('billapril', 'value'),
                State('billmay', 'value'),  
                State('billjune', 'value'),
                State('billjuly', 'value'),
                State('billaugust', 'value'),
                State('billseptember', 'value'),
                State('payapril', 'value'),
                State('paymay', 'value'),  
                State('payjune', 'value'),
                State('payjuly', 'value'),
                State('payaugust', 'value'),
                State('payseptember', 'value'),
                State('firstnamet', 'value'),
                State('lastnamet', 'value')
              ])
    
def update_output(clicks,*input_value):
    if clicks :
        #df.to_csv('Data_Tai.csv',index=False)
        
        data = pd.DataFrame([list(input_value)])
        data.columns = ['education', 'statut','repaySeptember','repayAugust','repayJuly','repayJune','repayMay','repayApril','limit', 'gender', 'age',
                       'billapril', 'billmay', 'billjune', 'billjuly', 'billaugust',
                       'billseptember', 'payapril', 'paymay', 'payjune', 'payjuly', 'payaugust',
                       'payseptember','lastnamet','firstnamet']
       
        data_final = np.zeros(78)
        taiwan=pd.DataFrame(data_final.reshape(1,-1),columns= ['university', 'high school', 'other_education', 'single',
       'other_mariage', 'PAY_01', 'PAY_02', 'PAY_03', 'PAY_04', 'PAY_05',
       'PAY_06', 'PAY_07', 'PAY_08', 'PAY_09', 'PAY_10', 'PAY_2_1', 'PAY_2_2',
       'PAY_2_3', 'PAY_2_4', 'PAY_2_5', 'PAY_2_6', 'PAY_2_7', 'PAY_2_8',
       'PAY_2_9', 'PAY_2_10', 'PAY_3_1', 'PAY_3_2', 'PAY_3_3', 'PAY_3_4',
       'PAY_3_5', 'PAY_3_6', 'PAY_3_7', 'PAY_3_8', 'PAY_3_9', 'PAY_3_10',
       'PAY_4_1', 'PAY_4_2', 'PAY_4_3', 'PAY_4_4', 'PAY_4_5', 'PAY_4_6',
       'PAY_4_7', 'PAY_4_8', 'PAY_4_9', 'PAY_4_10', 'PAY_5_1', 'PAY_5_2',
       'PAY_5_3', 'PAY_5_4', 'PAY_5_5', 'PAY_5_6', 'PAY_5_7', 'PAY_5_8',
       'PAY_5_9', 'PAY_6_1', 'PAY_6_2', 'PAY_6_3', 'PAY_6_4', 'PAY_6_5',
       'PAY_6_6', 'PAY_6_7', 'PAY_6_8', 'PAY_6_9', 'LIMIT_BAL', 'SEX', 'AGE',
       'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5',
       'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5',
       'PAY_AMT6'])
      
        taiwan["LIMIT_BAL"] = input_value[8]
        taiwan["SEX"]=input_value[9]
        taiwan["AGE"]= input_value[10]
        taiwan["BILL_AMT1"] = input_value[11]
        taiwan["BILL_AMT2"] = input_value[12]
        taiwan["BILL_AMT3"] = input_value[13]
        taiwan["BILL_AMT4"] = input_value[14]
        taiwan["BILL_AMT5"] = input_value[15]
        taiwan["BILL_AMT6"] = input_value[16]
        taiwan["PAY_AMT1"]  = input_value[17]
        taiwan["PAY_AMT2"]  = input_value[18]
        taiwan["PAY_AMT3"]  = input_value[19]
        taiwan["PAY_AMT4"]  = input_value[20]
        taiwan["PAY_AMT5"]  = input_value[21]
        taiwan["PAY_AMT6"]  = input_value[22]
        
        if input_value[0] != "graduate_school":
            taiwan[input_value[0]]=1
        if input_value[1] != "married":
            taiwan[input_value[1]]=1
        if input_value[2] != "PAY_00":
            taiwan[input_value[2]]=1
        if input_value[3] != "PAY_2_0":
            taiwan[input_value[3]]=1
        if input_value[4] != "PAY_3_0":
            taiwan[input_value[4]]=1
        if input_value[5] != "PAY_4_0":
            taiwan[input_value[5]]=1
        if input_value[6] != "PAY_5_0":
            taiwan[input_value[6]]=1
        if input_value[7] != "PAY_6_0":
            taiwan[input_value[7]]=1
        
        taiwan_Scaled = StandardScalerT.transform(taiwan)
        
        y_pred_knn_taiwan = model_KNN.predict(taiwan_Scaled)
        if y_pred_knn_taiwan==1:
            if input_value[9] == '1':
                return 'Monsieur '+input_value[23]+" "+input_value[24]+' est à 82% fiable'
            else:
                return 'Madame '+input_value[23]+" "+input_value[24]+' est à 82% fiable'
        else :
            if input_value[9] == '1':
                return 'Monsieur '+input_value[23]+" "+input_value[24]+' est à 82% non fiable'
            else:
                return 'Madame '+input_value[23]+" "+input_value[24]+' est à 82% non fiable'
    
@app.callback(  Output('output_div_am', 'children'),
                [Input('submit_america', 'n_clicks')],
                [State('reason', 'value'),
                State('job', 'value'),
                State('LOAN', 'value'),
                State('MORTDUE', 'value'),
                State('VALUE', 'value'),
                State('YOJ', 'value'),
                State('DEROG', 'value'),
                State('DELINQ', 'value'),
                State('CLAGE', 'value'),  
                State('NINQ', 'value'),
                State('CLNO', 'value'),
                State('DEBTINC', 'value'),
                State('firstnamea', 'value'),
                State('lastnamea', 'value')
                ])

def update_output_america(clicks,*input_value):
    if clicks :
        #df.to_csv('Data_Tai.csv',index=False)
        
        data = pd.DataFrame([list(input_value)])
        data.columns = ['reason', 'job', 'LOAN','MORTDUE', 'VALUE', 'YOJ', 'DEROG',
                       'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC','lastnamea','firstnamea']
       
        data_final = np.zeros(16)
        america=pd.DataFrame(data_final.reshape(1,-1),columns= ['LOAN', 'MORTDUE', 'VALUE', 'REASON', 'Mgr','Office', 'ProfExe', 'Sales', 'Other', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE','NINQ', 'CLNO', 'DEBTINC'])
    
        america["REASON"] = input_value[0] 
        america["LOAN"] = input_value[2]
        america["MORTDUE"]=input_value[3]
        america["VALUE"]= input_value[4]
        america["YOJ"] = input_value[5]
        america["DEROG"] = input_value[6]
        america["DELINQ"] =input_value[7]
        america["CLAGE"] = input_value[8]
        america["NINQ"] = input_value[9]
        america["CLNO"] =input_value[10]
        america["DEBTINC"] = input_value[11]
       
        
        if input_value[1] != "Self":
            america[input_value[1]]=1
       
        america_Scaled=StandardScalerA.transform(america)
        
        y_pred_knn_america =model_KNN_america.predict(america_Scaled)
        
        if y_pred_knn_america == 1:
            return 'Mr/Mme '+input_value[12]+" "+input_value[13]+' est à 91% fiable'
        else :
            return 'Mr/Mme '+input_value[12]+" "+input_value[13]+' est à 91% non fiable'


@app.callback(  Output('output_div_gr', 'children'),
                [Input('submit_german', 'n_clicks')],
                [State('Duration', 'value'),
                State('Credit amount', 'value'),
                State('Installment rate', 'value'),
                State('Present residence since', 'value'),
                State('Age in years', 'value'),
                State('Number of existing credits', 'value'),
                State('Number of people being liable to provide maintenance for', 'value'),
                State('Telephone', 'value'),
                State('Foreign worker', 'value'),
                State('Status', 'value'),
                State('Credit history', 'value'),
                State('Purpose', 'value'),
                State('Savings account', 'value'),
                State('Present employment', 'value'),
                State('Personal status', 'value'),
                State('Other debtors / guarantors', 'value'),
                State('Property', 'value'),
                State('Other installment plans', 'value'),
                State('Housing', 'value'),
                State('Job', 'value'),
                State('firstnameg', 'value'),
                State('lastnameg', 'value')
                ])

def update_output_german(clicks,*input_value):
    if clicks:
        
        data = pd.DataFrame([list(input_value)])
        data.columns = ['Duration','Credit amount', 'Installment rate', 'Present residence since','Age in years', 
                        'Number of existing credits', 'Number of people being liable to provide maintenance for',
                        'Telephone','Foreign worker', 'Status','Credit history', 'Purpose', 'Savings account', 'Present employment',
                        'Personal status','Other debtors / guarantors', 'Property', 'Other installment plans', 
                        'Housing', 'Job','lastnameg','firstnameg']
       
        data_final = np.zeros(47)
        german = pd.DataFrame(data_final.reshape(1,-1),columns= ['Duration', 'Credit amount', 'Installment rate',
       'Present residence since', 'Age in years', 'Number of existing credits',
       'Number of people being liable to provide maintenance for', 'Telephone',
       'Foreign worker', 'Status_A12', 'Status_A13', 'Status_A14',
       'Credit history_A31', 'Credit history_A32', 'Credit history_A33',
       'Credit history_A34', 'Purpose_A41', 'Purpose_A410', 'Purpose_A42',
       'Purpose_A43', 'Purpose_A44', 'Purpose_A45', 'Purpose_A46',
       'Purpose_A48', 'Purpose_A49', 'Savings account/bonds_A62',
       'Savings account/bonds_A63', 'Savings account/bonds_A64',
       'Savings account/bonds_A65', 'Present employment since_A72',
       'Present employment since_A73', 'Present employment since_A74',
       'Present employment since_A75', 'Personal status and sex_A92',
       'Personal status and sex_A93', 'Personal status and sex_A94',
       'Other debtors / guarantors_A102', 'Other debtors / guarantors_A103',
       'Property_A122', 'Property_A123', 'Property_A124',
       'Other installment plans_A142', 'Other installment plans_A143',
       'Housing_A152', 'Job_A172', 'Job_A173', 'Job_A174'])
    
        german["Duration"] = input_value[0] 
        german["Credit amount"] = input_value[1]
        german["Installment rate"]=input_value[2]
        german["Present residence since"]= input_value[3]
        german["Age in years"] = input_value[4]
        german["Number of existing credits"] = input_value[5]
        german["Number of people being liable to provide maintenance for"] =input_value[6]
        german["Telephone"] = input_value[7]
        german["Foreign worker"] = input_value[8]
       
        if input_value[9] != "Status_A11":
            german[input_value[9]]=1
        if input_value[10] != "Credit history_A30":
            german[input_value[10]]=1
        if input_value[11] != "Purpose_A40":
            german[input_value[11]]=1
        if input_value[12] != "Savings account/bonds_A61":
            german[input_value[12]]=1
        if input_value[13] != "Present employment since_A71":
            german[input_value[13]]=1
        if input_value[14] != "Personal status and sex_A91":
            german[input_value[14]]=1
        if input_value[15] != "Other debtors / guarantors_A101":
            german[input_value[15]]=1
        if input_value[16] != "Property_A121":
            german[input_value[16]]=1
        if input_value[17] != "Other installment plans_A141":
            german[input_value[17]]=1
        if input_value[18] != "Housing_A151":
            german[input_value[18]]=1
        if input_value[19] != "Job_A171":
            german[input_value[19]]=1
        
        german_Scaled=StandardScalerG.transform(german)
        
        #y_pred_knn_german =model_KNN_german.predict(german_Scaled)
        y_pred_SVM_german = svclassifier4.predict(german_Scaled)
        
        if y_pred_SVM_german == 0:
            return 'Mr/Mme '+input_value[20]+" "+input_value[21]+' est à 81% fiable'
        else :
            return 'Mr/Mme '+input_value[20]+" "+input_value[21]+' est à 81% non fiable'

@app.callback(Output('output-data-upload', 'children'),[Input('upload-data', 'contents'),Input('upload-data', 'filename')])
def update_table(contents, filename):
    table = html.Div()

    if contents:
        contents = contents[0]
        filename = filename[0]
        df = parse_data(contents, filename)

        table = html.Div([
            html.H5(filename),
            dash_table.DataTable(
                data=df.iloc[0:2,:].to_dict('rows'),
                columns=[{'name': i, 'id': i} for i in df.columns]
            ),
            html.Hr(),
            html.Div('Raw Content'),
            html.Pre(contents[0:200] + '...', style={
                'whiteSpace': 'pre-wrap',
                'wordBreak': 'break-all'
            })
        ])
        return table

@app.callback(dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),[dash.dependencies.Input('crossfilter-xaxis-column', 'value'),dash.dependencies.Input('crossfilter-yaxis-column', 'value'),dash.dependencies.Input('crossfilter-xaxis-type', 'value'),dash.dependencies.Input('crossfilter-yaxis-type', 'value'),dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,xaxis_type, yaxis_type,year_value):
    dff = df[df['BAD'] == year_value]

    return {'data': [dict(x=dff[dff['JOB'] == xaxis_column_name]['YOJ'],y=dff[dff['JOB'] == yaxis_column_name]['YOJ'],text=dff[dff['JOB'] == yaxis_column_name]['LOAN'],customdata=dff[dff['JOB'] == yaxis_column_name]['LOAN'],mode='markers',marker={'size': 15,'opacity': 0.5,'line': {'width': 0.5, 'color': 'white'}})],'layout': dict(xaxis={'title': xaxis_column_name,'type': 'linear' if xaxis_type == 'Linear' else 'log'},
             yaxis={'title': yaxis_column_name,'type': 'linear' if yaxis_type == 'Linear' else 'log'},
             margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
             height=450,
             hovermode='closest')}


def create_time_series(dff, axis_type, title):
    return {'data': [dict(x=dff['BAD'],y=dff['YOJ'],mode='lines+markers')],'layout': {'height': 225,'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},'annotations': [{'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom','xref': 'paper', 'yref': 'paper', 'showarrow': False,'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)','text': title}],'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},'xaxis': {'showgrid': False}}}


@app.callback(dash.dependencies.Output('x-time-series', 'figure'),[dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),dash.dependencies.Input('crossfilter-xaxis-column', 'value'),dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['LOAN'] == country_name]
    dff = dff[dff['JOB'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(dash.dependencies.Output('y-time-series', 'figure'),[dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),dash.dependencies.Input('crossfilter-yaxis-column', 'value'),dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['LOAN'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['JOB'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

    

In [26]:
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_table_experiments/dash_table_experiments.v0_6_0m1579306116.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_0m1590806334.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_0m1590806334.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_0m1590806334.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_0m1590806334.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_dash-component-suites/dash_core_components/highlight.v1_5_1m1590806334.pack.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2020 12:59:27] "GET /_

In [ ]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, State, Input



app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id='username', value='Initial Value', type='text'),
    html.Button(id='submit-button', type='submit', children='Submit'),
    html.Div(id='output_div')
])

@app.callback(Output('output_div', 'children'),[Input('submit-button', 'n_clicks')],[State('username', 'value')])
def update_output(clicks,input_value):
    if clicks is not None:
        clicks, input_value

In [ ]:
app.run_server()

In [ ]:
    
     html.Label('Amount of the loan request',
               style={'width': '40%','margin-top':"15px",'margin-left':'10px'}),
    dcc.Slider(
        min=0,
        max=5,
        marks={i: ''.format(i) if i == 1 else str(i) for i in range(1, 5)},
        value=1,
    ),
    
    dcc.Dropdown(
        value='Choose your bank',
        style={'width':'63%','margin-top':"15px",'margin-left':'5px'},
        placeholder='Choose your Reason',
        options=[
            {'label': 'Reason', 'value': 'Reason'},
            {'label': 'Home improvement', 'value': 'HomeImp'},
            {'label': 'Debt consolidation', 'value': 'DebtCon'}
        ]
    ),
                

In [ ]:
dcc.Graph(
        id='life-exp-vs-gdp',
        figure={
            'data': [
                dict(
                    x=df[df['LIMIT_BAL'] == i]['single'],
                    y=df[df['LIMIT_BAL'] == i]['high school'],
                    text=df[df['LIMIT_BAL'] == i]['university'],
                    mode='markers',
                    opacity=0.7,
                    marker={
                        'size': 15,
                        'line': {'width': 0.5, 'color': 'white'}
                    },
                    name=i
                ) for i in df.LIMIT_BAL.unique()
            ],
            'layout': dict(
                xaxis={'type': 'log', 'title': 'single'},
                yaxis={'title': 'high school'},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    )

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, State, Input

if __name__ == '__main__':
    app = dash.Dash()

    app.layout = html.Div([
        dcc.Input(id='username', value='12', type='text'),
        html.Button(id='submit-button', type='submit', children='Submit'),
        html.Div(id='output_div')
    ])

    @app.callback(Output('output_div', 'children'),
                  [Input('submit-button', 'n_clicks')],
                  [State('username', 'value')],
                  )
    
    def update_output(clicks, input_value):
        if clicks is not None:
            return  clicks, input_value



In [ ]:
    app.run_server()

In [ ]:
import base64
import datetime
import io
import dash_table
import plotly.graph_objs as go
#import cufflinks as cf

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table

import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

colors = {
    "graphBackground": "#F5F5F5",
    "background": "#ffffff",
    "text": "#000000"
}

app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    dcc.Graph(id='Mygraph'),
    html.Div(id='output-data-upload')
])

def parse_data(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV or TXT file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'txt' or 'tsv' in filename:
            # Assume that the user upl, delimiter = r'\s+'oaded an excel file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')), delimiter = r'\s+')
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return df


@app.callback(Output('Mygraph', 'figure'),
            [
                Input('upload-data', 'contents'),
                Input('upload-data', 'filename')
            ])
def update_graph(contents, filename):
    fig = {
        'layout': go.Layout(
            plot_bgcolor=colors["graphBackground"],
            paper_bgcolor=colors["graphBackground"])
    }

    if contents:
        contents = contents[0]
        filename = filename[0]
        df = parse_data(contents, filename)
        df = df.set_index(df.columns[0])
        fig['data'] = df.iplot(asFigure=True, kind='scatter', mode='lines+markers', size=1)


    return fig

@app.callback(Output('output-data-upload', 'children'),
            [
                Input('upload-data', 'contents'),
                Input('upload-data', 'filename')
            ])
def update_table(contents, filename):
    table = html.Div()

    if contents:
        contents = contents[0]
        filename = filename[0]
        df = parse_data(contents, filename)

        table = html.Div([
            html.H5(filename),
            dash_table.DataTable(
                data=df.to_dict('rows'),
                columns=[{'name': i, 'id': i} for i in df.columns]
            ),
            html.Hr(),
            html.Div('Raw Content'),
            html.Pre(contents[0:200] + '...', style={
                'whiteSpace': 'pre-wrap',
                'wordBreak': 'break-all'
            })
        ])

    return table

In [ ]:
app.run_server()